In [37]:
# Import libraries
import pandas as pd

In [38]:
# Import the chrome data
chrome_df = pd.read_csv("../CB_chrome_cleaned.csv")
chrome_df = chrome_df.drop(columns=["Unnamed: 0"])
chrome_df = chrome_df.rename(columns={"url": "url_chrome", "title": "title_chrome","visit_time": "time", "visit_date": "date"})

chrome_df["date"] = pd.to_datetime(chrome_df["date"])
start_date = "2025-01-01"
end_date = "2025-02-17"
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)
chrome_df = chrome_df[(chrome_df["date"] >= start_date) & (chrome_df["date"] <= end_date)]
chrome_df = chrome_df.sort_values("date")

chrome_df.head()

,url_chrome,visit_count,typed_count,title_chrome,time,date
15250,https://www.google.com/search?q=dsw&rlz=1C1ONG...,2,0,dsw - Google Search,02:05:20,2025-01-01
15407,https://www.google.com/search?q=tysons&rlz=1C1...,2,0,tysons - Google Search,22:47:12,2025-01-01
15408,https://www.google.com/search?q=tysons&rlz=1C1...,2,0,tysons - Google Search,22:47:12,2025-01-01
15409,https://www.tysonscornercenter.com/,1,0,Tysons Corner Center | Home,22:47:13,2025-01-01
15410,https://www.tysonscornercenter.com/Dining,3,0,Tysons Corner Center | Dining and Refreshments,22:47:21,2025-01-01


In [39]:
# get the first and last visited websites per date

def get_first_and_last_visits(df):
    # find first and last visited websites per date
    first_visits = df.loc[df.groupby("date")["time"].idxmin()]
    last_visits = df.loc[df.groupby("date")["time"].idxmax()]

    # select relevant columns
    first_visits = first_visits[["date", "time", "url_chrome", "title_chrome"]].rename(columns={
        "time": "chrome_visit_time_first",
        "url_chrome": "chrome_url_first",
        "title_chrome": "chrome_title_first"
    })
    
    last_visits = last_visits[["date", "time", "url_chrome", "title_chrome"]].rename(columns={
        "time": "chrome_visit_time_last",
        "url_chrome": "chrome_url_last",
        "title_chrome": "chrome_title_last"
    })

    # merge both DataFrames on visit_date to get one row per date
    visit_summary = pd.merge(first_visits, last_visits, on="date")
    return visit_summary

visit_summary_df = get_first_and_last_visits(chrome_df)
visit_summary_df.tail()

,date,chrome_visit_time_first,chrome_url_first,chrome_title_first,chrome_visit_time_last,chrome_url_last,chrome_title_last
38,2025-02-10,03:23:32,https://docs.google.com/presentation/?usp=slid...,Google Slides,23:55:24,https://www.netflix.com/watch/81773841?trackId...,Netflix
39,2025-02-11,02:53:29,https://docs.google.com/spreadsheets/u/1/d/e/2...,CS4774 ML Schedule - Google Drive,23:49:30,https://chatgpt.com/c/67abe207-6820-800b-99fc-...,Logarithmic Expression Simplification
40,2025-02-12,01:36:19,https://www.google.com/search?q=gmail&rlz=1C1O...,gmail - Google Search,23:53:21,https://www.youtube.com/watch?v=6v_R180kIGs,HALF HORSE HALF MAN | OFFICIAL VIDEO - YouTube
41,2025-02-13,00:49:50,https://www.google.com/search?q=problems+you+c...,problems you can solve in prolog - Google Search,23:57:44,https://support.google.com/accounts/answer/302...,How to download your Google data - Google Acco...
42,2025-02-14,00:13:53,https://www.google.com/search?q=hrome%3A%2F%2F...,chrome://settings/syncSetup - Google Search,00:21:53,https://sqlitebrowser.org/dl/,Downloads - DB Browser for SQLite


In [40]:
# Import the youtube and sleep data
youtube_sleep_df = pd.read_csv("../youtube_sleep_CB.csv")
youtube_sleep_df = youtube_sleep_df.drop(columns=["Unnamed: 0", "time_first", "url_first", "title_first", "type_first", "time_last", "url_last", "title_last", "type_last", "date_time", "session_id", "session_duration"])
youtube_sleep_df["date"] = pd.to_datetime(youtube_sleep_df["date"])
youtube_sleep_df.head()


,date,total_sleep_hours,sleep_start,sleep_end,had_session,total_videos_watched
0,2025-01-07,8.816667,2025-01-07 00:51:00-05:00,2025-01-07 10:00:00-05:00,True,20
1,2025-01-09,6.200000,2025-01-09 00:47:51-05:00,2025-01-09 07:44:51-05:00,True,4
2,2025-01-22,7.025000,2025-01-22 01:54:51-05:00,2025-01-22 08:59:51-05:00,True,6
3,2025-02-06,6.883333,2025-02-06 03:05:30-05:00,2025-02-06 10:00:00-05:00,True,5
4,2025-02-08,7.216667,2025-02-08 02:51:47-05:00,2025-02-08 12:48:25-05:00,True,2


In [41]:
df = pd.merge(youtube_sleep_df, visit_summary_df, on="date", how="inner")
df.head()

,date,total_sleep_hours,sleep_start,sleep_end,had_session,total_videos_watched,chrome_visit_time_first,chrome_url_first,chrome_title_first,chrome_visit_time_last,chrome_url_last,chrome_title_last
0,2025-01-07,8.816667,2025-01-07 00:51:00-05:00,2025-01-07 10:00:00-05:00,True,20,02:00:37,https://www.amazon.com/cart?ref=sc_refresh_glow,Amazon.com Shopping Cart,23:54:39,https://mail.google.com/mail/u/0/?tab=rm&ogbl#...,"Inbox (6,286) - eqk9vb@virginia.edu - UVa Mail"
1,2025-01-09,6.200000,2025-01-09 00:47:51-05:00,2025-01-09 07:44:51-05:00,True,4,00:30:18,https://www.amazon.com/,Amazon.com. Spend less. Smile more.,23:37:06,https://na2.docusign.net/Signing/SessionTimeou...,Your session has timed out.
2,2025-01-22,7.025000,2025-01-22 01:54:51-05:00,2025-01-22 08:59:51-05:00,True,6,02:44:40,https://canvas.its.virginia.edu/courses/132927,Responsible AI,23:15:04,https://canvas.its.virginia.edu/courses/136546...,Week 5: The Police: STS 4600 Sp 25 Sec 011
3,2025-02-06,6.883333,2025-02-06 03:05:30-05:00,2025-02-06 10:00:00-05:00,True,5,00:00:34,https://likenew.lululemon.com/shop?productFilt...,New Arrivals | lululemon | lululemon Like New,23:58:46,https://docs.google.com/forms/d/12wwzIYs5Ey6VL...,[STUDENT] Survey on Ethics of ChatGPT in Educa...
4,2025-02-08,7.216667,2025-02-08 02:51:47-05:00,2025-02-08 12:48:25-05:00,True,2,02:53:03,https://docs.google.com/presentation/?usp=slid...,Google Slides,21:41:11,https://likenew.lululemon.com/productStyle/hot...,"Hotty Hot High-Rise Short 4"" Lined - Resale | ..."


In [42]:
keywords = pd.read_csv("../CB_data/cheryl_uva_keywords_chrome.csv")
keywords = keywords.drop(columns=["Unnamed: 0"])
keywords = keywords.rename(columns={"visit_date": "date"})
keywords["date"] = pd.to_datetime(keywords["date"])
df = pd.merge(keywords, df, on="date",how="inner")
df.head()

,date,total_count,uva,canvas,eqk9vb,drive,last_time,total_sleep_hours,sleep_start,sleep_end,had_session,total_videos_watched,chrome_visit_time_first,chrome_url_first,chrome_title_first,chrome_visit_time_last,chrome_url_last,chrome_title_last
0,2025-01-07,292,234,0,47,11,2025-02-23 23:54:39,8.816667,2025-01-07 00:51:00-05:00,2025-01-07 10:00:00-05:00,True,20,02:00:37,https://www.amazon.com/cart?ref=sc_refresh_glow,Amazon.com Shopping Cart,23:54:39,https://mail.google.com/mail/u/0/?tab=rm&ogbl#...,"Inbox (6,286) - eqk9vb@virginia.edu - UVa Mail"
1,2025-01-09,139,56,0,11,72,2025-02-23 23:37:06,6.200000,2025-01-09 00:47:51-05:00,2025-01-09 07:44:51-05:00,True,4,00:30:18,https://www.amazon.com/,Amazon.com. Spend less. Smile more.,23:37:06,https://na2.docusign.net/Signing/SessionTimeou...,Your session has timed out.
2,2025-01-22,174,87,53,15,19,2025-02-23 23:15:04,7.025000,2025-01-22 01:54:51-05:00,2025-01-22 08:59:51-05:00,True,6,02:44:40,https://canvas.its.virginia.edu/courses/132927,Responsible AI,23:15:04,https://canvas.its.virginia.edu/courses/136546...,Week 5: The Police: STS 4600 Sp 25 Sec 011
3,2025-02-06,173,25,61,7,80,2025-02-23 23:58:46,6.883333,2025-02-06 03:05:30-05:00,2025-02-06 10:00:00-05:00,True,5,00:00:34,https://likenew.lululemon.com/shop?productFilt...,New Arrivals | lululemon | lululemon Like New,23:58:46,https://docs.google.com/forms/d/12wwzIYs5Ey6VL...,[STUDENT] Survey on Ethics of ChatGPT in Educa...
4,2025-02-08,127,51,5,14,57,2025-02-23 21:41:11,7.216667,2025-02-08 02:51:47-05:00,2025-02-08 12:48:25-05:00,True,2,02:53:03,https://docs.google.com/presentation/?usp=slid...,Google Slides,21:41:11,https://likenew.lululemon.com/productStyle/hot...,"Hotty Hot High-Rise Short 4"" Lined - Resale | ..."


In [43]:
df['day_of_week'] = df['date'].dt.day_name()


In [44]:
df.to_csv("cheryl_data.csv")